In [3]:
import os

# Add the directory containing the executable to the PATH
os.environ["PATH"] += os.pathsep + "/usr/local/cuda/bin"

# Check if the directory is added to the a
print(os.environ["PATH"])

/opt/tljh/user/bin:/bin:/usr/bin:/usr/local/cuda/bin


In [4]:
%%bash
nvcc --version
nvprof --version
nsys --version
ncu --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:24:57_PDT_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
nvprof: NVIDIA (R) Cuda command line profiler
Copyright (c) 2012 - 2025 NVIDIA Corporation
Release version 12.9.19 (21)
NVIDIA Nsight Systems version 2025.1.3.140-251335620677v0
NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2025 NVIDIA Corporation
Version 2025.2.0.0 (build 35613519) (public-release)


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov  9 04:20:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-32GB           Off |   00000000:00:10.0 Off |                    0 |
| N/A   27C    P0             22W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
%%writefile kmeans_ann_cuda_search.cu 

#include <cuda_runtime.h>
#include <cstdio>
#include <vector>
#include <random>
#include <algorithm>
#include <numeric>
#include <cmath>
#include <cassert>
#include <unordered_set>
#include <limits>
#include <cfloat>

#include <thrust/device_ptr.h>
#include <thrust/device_vector.h>
#include <thrust/sort.h>
#include <thrust/sequence.h>
#include <thrust/copy.h>

#include <cub/cub.cuh>

// ---------------- Config ----------------
static const int DIM          = 64;
static const float ALPHA      = 0.7f;
static const int SEED         = 2025; 
static const int N            = 1 << 20; // demo: 1M 160m
static const int K            = 1024;
static const int NPROBE       = 4;       // cluster depth during search
static const int TOPK         = 5;

static const int KMEANS_ITERS = 15;      // K-Means iters

using Vec = std::vector<float>;

struct Pair { float key; int id; };

// ---------------- Embedding basic ----------------

static Vec numberBase[76]; // 1..75
static Vec posBase[25];    // 0..24

static void normInPlace(Vec &v) {
    double s = 0;
    for (float x : v) s += (double)x * x;
    float n = float(std::sqrt(s) + 1e-12);
    for (float &x : v) x /= n;
}

static Vec randUnit(std::mt19937 &rng) {
    std::uniform_real_distribution<float> U(-1.f, 1.f);
    Vec v(DIM);
    for (int i = 0; i < DIM; i++) v[i] = U(rng);
    normInPlace(v);
    return v;
}

static void initBases() {
    std::mt19937 rng(SEED);
    for (int n = 1; n <= 75; n++) numberBase[n] = randUnit(rng);
    for (int i = 0; i < 25; i++)  posBase[i]    = randUnit(rng);
}

static Vec cardToVec(const int card[25]) {
    Vec out(DIM, 0.f);
    for (int i = 0; i < 25; i++) {
        int n = card[i];
        if (n < 1 || n > 75) {
            fprintf(stderr, "number out of range\n");
            exit(1);
        }
        const Vec &b = numberBase[n];
        const Vec &p = posBase[i];
        for (int j = 0; j < DIM; j++)
            out[j] += b[j] + ALPHA * p[j];
    }
    normInPlace(out);
    return out;
}

static void genCard(std::mt19937 &rng, int out[25]) {
    std::vector<int> p(75);
    std::iota(p.begin(), p.end(), 1);
    std::shuffle(p.begin(), p.end(), rng);
    for (int i = 0; i < 25; i++) out[i] = p[i];
}

static double dot_host(const float* a, const float* b) {
    double s = 0;
    for (int i = 0; i < DIM; i++) s += (double)a[i] * b[i];
    return s;
}

// ---------------- Device: Distance Kernels ----------------

// data[0..Ntot-1] vs single q
__global__ void cosineDistKernel(const float* data,
                                 const float* q,
                                 int Ntot,
                                 float* out) {
    for (int i = blockIdx.x * blockDim.x + threadIdx.x;
         i < Ntot;
         i += gridDim.x * blockDim.x) {
        const float* row = data + (size_t)i * DIM;
        float dot = 0.f;
        for (int d = 0; d < DIM; ++d) dot += row[d] * q[d];
        out[i] = 1.f - dot;
    }
}

// assign idxSel[0..M-1] vs q
__global__ void cosineDistIndexKernel(const float* data,
                                      const float* q,
                                      const int* idxSel,
                                      int M,
                                      float* distSel) {
    for (int j = blockIdx.x * blockDim.x + threadIdx.x;
         j < M;
         j += gridDim.x * blockDim.x) {
        int i = idxSel[j];
        const float* row = data + (size_t)i * DIM;
        float dot = 0.f;
        for (int d = 0; d < DIM; ++d) dot += row[d] * q[d];
        distSel[j] = 1.f - dot;
    }
}

// ----------------  GPU K-Means Kernels ----------------

// E-step + accumulate:
// every data to search nearest center -> assign[i]
// atomicAdd sums[c][d] and counts[c]
__global__ void assignAndAccumulateKernel(const float* data,
                                          int N,
                                          const float* centroids,
                                          int K,
                                          int* assign,
                                          float* sums,
                                          int* counts) {
    for (int i = blockIdx.x * blockDim.x + threadIdx.x;
         i < N;
         i += gridDim.x * blockDim.x) {

        const float* xi = data + (size_t)i * DIM;

        int bestC = 0;
        float bestD = 1e30f;

        for (int c = 0; c < K; ++c) {
            const float* ctr = centroids + (size_t)c * DIM;
            float dot = 0.f;
            for (int d = 0; d < DIM; ++d) {
                dot += xi[d] * ctr[d];
            }
            float dist = 1.f - dot;
            if (dist < bestD) {
                bestD = dist;
                bestC = c;
            }
        }

        assign[i] = bestC;

        atomicAdd(&counts[bestC], 1);
        size_t base = (size_t)bestC * DIM;
        for (int d = 0; d < DIM; ++d) {
            atomicAdd(&sums[base + d], xi[d]);
        }
    }
}

// M-step: sums/counts -> update centroid + norm
__global__ void updateCentroidsKernel(float* centroids,
                                      const float* sums,
                                      const int* counts,
                                      int K) {
    int c = blockIdx.x * blockDim.x + threadIdx.x;
    if (c >= K) return;

    int cnt = counts[c];
    float* ctr = centroids + (size_t)c * DIM;
    const float* sumc = sums + (size_t)c * DIM;

    if (cnt > 0) {
        double norm2 = 0.0;
        for (int d = 0; d < DIM; ++d) {
            float v = sumc[d] / (float)cnt;
            ctr[d] = v;
            norm2 += (double)v * (double)v;
        }
        float n = float(std::sqrt(norm2) + 1e-12);
        for (int d = 0; d < DIM; ++d) {
            ctr[d] /= n;
        }
    }
}

// ---------------- Host: Inverted Lists (vector<vector<int>>) ----------------

static void buildInvertedLists(
    const std::vector<int>& assign,
    int N, int K,
    std::vector<std::vector<int>>& lists
) {
    lists.assign(K, {});
    for (int i = 0; i < N; ++i) {
        int c = assign[i];
        if (c >= 0 && c < K) {
            lists[c].push_back(i);
        }
    }
}

static void buildCSRFromLists(
    const std::vector<std::vector<int>>& lists,
    int K, int N,
    std::vector<int>& listOffsets,
    std::vector<int>& listIds
) {
    listOffsets.assign(K + 1, 0);
    listIds.clear();
    listIds.reserve(N);

    int offset = 0;
    for (int c = 0; c < K; ++c) {
        listOffsets[c] = offset;
        const std::vector<int>& bucket = lists[c];
        listIds.insert(listIds.end(), bucket.begin(), bucket.end());
        offset += (int)bucket.size();
    }
    listOffsets[K] = offset;
}

// ---------------- Device: gatherCandidates ----------------

// centIds:     
// nprobe:      nprobe to dect
// listOffsets: CSR offsets,
// listIds:     CSR ids
// outIdx:     
// outCount:    
__global__ void gatherCandidatesKernel(
    const int* centIds,
    int nprobe,
    const int* listOffsets,
    const int* listIds,
    int* outIdx,
    int* outCount
) {
    int p = blockIdx.x * blockDim.x + threadIdx.x;
    if (p >= nprobe) return;

    int c = centIds[p];
    int start = listOffsets[c];
    int end   = listOffsets[c + 1];
    int len   = end - start;
    if (len <= 0) return;

    int base = atomicAdd(outCount, len);
    for (int i = 0; i < len; ++i) {
        outIdx[base + i] = listIds[start + i];
    }
}

// ---------------- Main ----------------
int main() {

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);

    int threads = prop.maxThreadsPerBlock;  
    int BLOCK  = prop.multiProcessorCount; 
    printf("Params: N=%d  K=%d  nprobe=%d  TOPK=%d  DIM=%d  KMEANS_ITERS=%d threads=%d blocks=%d\n",
           N, K, NPROBE, TOPK, DIM, KMEANS_ITERS,threads,BLOCK);

    initBases();
    std::mt19937 rng(SEED + 7);

    // 1) building dataset
    std::vector<int>   h_cards((size_t)N * 25);
    std::vector<float> h_data((size_t)N * DIM);

    for (int i = 0; i < N; i++) {
        int c[25];
        genCard(rng, c);
        for (int t = 0; t < 25; ++t)
            h_cards[(size_t)i * 25 + t] = c[t];
        Vec v = cardToVec(c);
        for (int d = 0; d < DIM; ++d)
            h_data[(size_t)i * DIM + d] = v[d];
    }

    // 2) build query
    int qc[25];
    for (int t = 0; t < 25; ++t) qc[t] = h_cards[t];
    qc[3] = 75; qc[17] = 1; std::swap(qc[5], qc[19]);
    Vec qvec = cardToVec(qc);

    {
        std::vector<float> qhost(qvec.begin(), qvec.end());
        std::vector<float> d0(DIM);
        for (int i = 0; i < DIM; i++) d0[i] = h_data[i];
        double dot0 = dot_host(qhost.data(), d0.data());
        printf("[DEBUG #1] host dot(q, data[0])=%.9f  dist=%.9f\n",
               dot0, 1.0 - dot0);
    }

    // 3) GPU buffers
    float *d_data = 0, *d_q = 0, *d_centroids = 0;
    float *d_tmpCentDist = 0;

    cudaMalloc(&d_data, (size_t)N * DIM * sizeof(float));
    cudaMalloc(&d_q, DIM * sizeof(float));
    cudaMalloc(&d_centroids, (size_t)K * DIM * sizeof(float));
    cudaMalloc(&d_tmpCentDist, K * sizeof(float));

    cudaMemcpy(d_data, h_data.data(),
               (size_t)N * DIM * sizeof(float),
               cudaMemcpyHostToDevice);
    cudaMemcpy(d_q, qvec.data(),
               DIM * sizeof(float),
               cudaMemcpyHostToDevice);

    // 4) initial centroids: Host random select K centers
    {
        std::vector<int> idx(N);
        std::iota(idx.begin(), idx.end(), 0);
        std::shuffle(idx.begin(), idx.end(), rng);
        std::vector<float> h_initC((size_t)K * DIM);
        for (int c = 0; c < K; ++c) {
            int i = idx[c];
            std::copy_n(&h_data[(size_t)i * DIM], DIM,
                        &h_initC[(size_t)c * DIM]);
        }
        cudaMemcpy(d_centroids, h_initC.data(),
                   (size_t)K * DIM * sizeof(float),
                   cudaMemcpyHostToDevice);
    }

    int   *d_assign = 0;
    float *d_sums   = 0;
    int   *d_counts = 0;

    cudaMalloc(&d_assign, N * sizeof(int));
    cudaMalloc(&d_sums,   (size_t)K * DIM * sizeof(float));
    cudaMalloc(&d_counts, K * sizeof(int));

    // 5) GPU K-Means
    {
        dim3 block(BLOCK);
        dim3 gridN((N + BLOCK - 1) / BLOCK);
        dim3 gridK((K + BLOCK - 1) / BLOCK);

        printf("[BUILD] GPU K-Means: iters=%d\n", KMEANS_ITERS);

        for (int it = 0; it < KMEANS_ITERS; ++it) {
            cudaMemset(d_sums,   0, (size_t)K * DIM * sizeof(float));
            cudaMemset(d_counts, 0, K * sizeof(int));

            assignAndAccumulateKernel<<<gridN, block>>>(
                d_data, N, d_centroids, K,
                d_assign, d_sums, d_counts
            );
            cudaDeviceSynchronize();

            updateCentroidsKernel<<<gridK, block>>>(
                d_centroids, d_sums, d_counts, K
            );
            cudaDeviceSynchronize();
        }
    }

    // 6) build inverted index on host
    std::vector<int> h_assign(N);
    cudaMemcpy(h_assign.data(), d_assign,
               N * sizeof(int),
               cudaMemcpyDeviceToHost);

    // IVF FOR 
    std::vector<std::vector<int>> lists;
    buildInvertedLists(h_assign, N, K, lists);

    int nonEmpty = 0;
    for (int c = 0; c < K; ++c)
        if (!lists[c].empty()) nonEmpty++;
    printf("[BUILD] Non-empty clusters: %d / %d\n", nonEmpty, K);

    // 7) flatten lists -> CSR, copy to device
    std::vector<int> h_listOffsets;
    std::vector<int> h_listIds;
    buildCSRFromLists(lists, K, N, h_listOffsets, h_listIds);

    int *d_listOffsets = 0;
    int *d_listIds     = 0;
    cudaMalloc(&d_listOffsets, (K + 1) * sizeof(int));
    cudaMalloc(&d_listIds,     h_listIds.size() * sizeof(int));

    cudaMemcpy(d_listOffsets, h_listOffsets.data(),
               (K + 1) * sizeof(int),
               cudaMemcpyHostToDevice);
    cudaMemcpy(d_listIds, h_listIds.data(),
               h_listIds.size() * sizeof(int),
               cudaMemcpyHostToDevice);

    // 8) Search Stage 1: q vs centroids -> Thrust sort for NPROBE
    {
        dim3 block(BLOCK);
        dim3 grid((K + BLOCK - 1) / BLOCK);
        cosineDistKernel<<<grid, block>>>(d_centroids, d_q, K, d_tmpCentDist);
        cudaDeviceSynchronize();
    }

    thrust::device_ptr<float> dist_ptr(d_tmpCentDist);
    thrust::device_vector<int> d_centIds(K);
    thrust::sequence(d_centIds.begin(), d_centIds.end()); // 0..K-1

    thrust::sort_by_key(dist_ptr, dist_ptr + K, d_centIds.begin());

    // 9) GPU  collect the NPROBE numbers of cluster to do candiating
    int *d_outIdx   = 0;
    int *d_outCount = 0;
    cudaMalloc(&d_outIdx,   N * sizeof(int));  // 上界 N（通常远小于）
    cudaMalloc(&d_outCount, sizeof(int));
    cudaMemset(d_outCount,  0, sizeof(int));

    {
        gatherCandidatesKernel<<<BLOCK, threads>>>(
            thrust::raw_pointer_cast(d_centIds.data()),
            NPROBE,
            d_listOffsets,
            d_listIds,
            d_outIdx,
            d_outCount
        );
        cudaDeviceSynchronize();
    }

    int M = 0;
    cudaMemcpy(&M, d_outCount, sizeof(int), cudaMemcpyDeviceToHost);
    printf("Candidates from chosen centers: M=%d (of N=%d)\n", M, N);

    if (M <= 0) {
        printf("No candidates, abort.\n");
    } else {
        // 10) Stage 2: candidate Top-K on GPU via CUB

        float *d_selDist   = 0;   // distance between candidates
        float *d_keys_out  = 0;   // distance after sorting
        int   *d_vals_out  = 0;   // candidates id after sorting
        void  *d_temp_storage = 0;
        size_t temp_bytes = 0;
        
        // 1) 为距离分配空间
        cudaMalloc(&d_selDist, M * sizeof(float));
        
        // 2) 计算 query vs 每个候选向量的距离
        {
            dim3 block(BLOCK);
            dim3 grid((M + BLOCK - 1) / BLOCK);
            // 注意这里直接用 d_outIdx 作为候选 id 列表
            cosineDistIndexKernel<<<grid, block>>>(
                d_data, d_q, d_outIdx, M, d_selDist
            );
            cudaDeviceSynchronize();
        }
        
        // 3)CUB malloc space
        cudaMalloc(&d_keys_out, M * sizeof(float));
        cudaMalloc(&d_vals_out, M * sizeof(int));
        
        // 4) first time using SortPairs  to access size of buffer ares needed
        cub::DeviceRadixSort::SortPairs(
            d_temp_storage, temp_bytes,
            d_selDist, d_keys_out,   // keys: distance
            d_outIdx,  d_vals_out,   // values: gathered id（
            M
        );
        cudaMalloc(&d_temp_storage, temp_bytes);
        
        // 5) really sort from nearest to further
        cub::DeviceRadixSort::SortPairs(
            d_temp_storage, temp_bytes,
            d_selDist, d_keys_out,
            d_outIdx,  d_vals_out,
            M
        );
        cudaDeviceSynchronize();
        
        // 6) extract top to CPU，做 TopK & 打印
        int take = std::min(M, TOPK * 4);
        std::vector<float> h_keys(take);
        std::vector<int>   h_ids(take);
        
        cudaMemcpy(h_keys.data(), d_keys_out,
                   take * sizeof(float),
                   cudaMemcpyDeviceToHost);
        cudaMemcpy(h_ids.data(), d_vals_out,
                   take * sizeof(int),
                   cudaMemcpyDeviceToHost);
        
        std::vector<Pair> uniq;
        uniq.reserve(TOPK);
        std::unordered_set<int> seen;
        
        for (int i = 0; i < take; ++i) {
            int id = h_ids[i];
            float dist = h_keys[i];
            if (id < 0 || !std::isfinite(dist)) continue;
            if (seen.insert(id).second) {
                uniq.push_back(Pair{dist, id});
                if ((int)uniq.size() == TOPK) break;
            }
        }

        printf("Top-%d among %d candidates (CUB SortPairs):\n",
               (int)uniq.size(), M);
        for (int i = 0; i < (int)uniq.size(); ++i) {
            printf("%2d) id=%d  dist=%.6f  sim=%.6f\n",
                   i+1, uniq[i].id, uniq[i].key, 1.f - uniq[i].key);
        }

        if (!uniq.empty()) {
            int bestId = uniq[0].id;
            std::vector<float> qhost(qvec.begin(), qvec.end());
            std::vector<float> dv(DIM);
            for (int i = 0; i < DIM; ++i)
                dv[i] = h_data[(size_t)bestId * DIM + i];
            double dotChk = dot_host(qhost.data(), dv.data());
            printf("[DEBUG] host check best id=%d  dot=%.9f  dist=%.9f\n",
                   bestId, dotChk, 1.0 - dotChk);
        }

        cudaFree(d_temp_storage);
        cudaFree(d_keys_out);
        cudaFree(d_vals_out);
        cudaFree(d_selDist);
    }

    // 11) free
    cudaFree(d_outIdx);
    cudaFree(d_outCount);
    cudaFree(d_tmpCentDist);
    cudaFree(d_centroids);
    cudaFree(d_q);
    cudaFree(d_data);
    cudaFree(d_assign);
    cudaFree(d_sums);
    cudaFree(d_counts);
    cudaFree(d_listOffsets);
    cudaFree(d_listIds);

    return 0;
}



Overwriting kmeans_ann_cuda_search.cu


In [7]:
%%bash
nvcc -O3 -std=c++17 -arch=sm_70 kmeans_ann_cuda_search.cu -o kmeans_ann_cuda_search -Wno-deprecated-gpu-targets

In [18]:
%%bash
nvprof ./kmeans_ann_cuda_search

==1111200== NVPROF is profiling process 1111200, command: ./kmeans_ann_cuda_search


Params: N=1048576  K=1024  nprobe=4  TOPK=5  DIM=64  KMEANS_ITERS=15 threads=1024 blocks=80
[DEBUG #1] host dot(q, data[0])=0.927471359  dist=0.072528641
[BUILD] GPU K-Means: iters=15
[BUILD] Non-empty clusters: 1024 / 1024
Candidates from chosen centers: M=4655 (of N=1048576)
Top-5 among 4655 candidates (CUB SortPairs):
 1) id=89698  dist=0.157913  sim=0.842087
 2) id=913782  dist=0.158968  sim=0.841032
 3) id=350035  dist=0.164414  sim=0.835586
 4) id=56801  dist=0.177664  sim=0.822336
 5) id=91744  dist=0.177767  sim=0.822233
[DEBUG] host check best id=89698  dot=0.842086914  dist=0.157913086


==1111200== Profiling application: ./kmeans_ann_cuda_search
==1111200== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   70.57%  666.16ms        15  44.411ms  43.935ms  45.312ms  assignAndAccumulateKernel(float const *, int, float const *, int, int*, float*, int*)
                   28.93%  273.07ms         5  54.615ms  1.0880us  269.68ms  [CUDA memcpy HtoD]
                    0.45%  4.2173ms         4  1.0543ms  1.9520us  4.2092ms  [CUDA memcpy DtoH]
                    0.03%  315.77us        15  21.051us  19.999us  24.672us  updateCentroidsKernel(float*, float const *, int const *, int)
                    0.01%  115.71us         1  115.71us  115.71us  115.71us  gatherCandidatesKernel(int const *, int, int const *, int const *, int*, int*)
                    0.00%  47.136us        31  1.5200us     672ns  2.5600us  [CUDA memset]
                    0.00%  24.832us         1  24.832us  24.832us  24.832us  void cub: